In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API credentials using the JSON module
with open('/Users/rdbha/.secret/yelp_api.json') as f: 
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [4]:
# set our API call parameters and filename before the first call
LOCATION = 'Greenville, SC'
TERM = 'sushi'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = f"Data/results_in_progress_sushi.json"
JSON_FILE

'Data/results_in_progress_sushi.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_sushi.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

99

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

5

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/5 [00:00<?, ?it/s]

In [14]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,zG_XOAFi9Y560WJ1RvghBw,sushi-masa-japanese-restaurant-greenville,Sushi-Masa Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/zsRavZ...,False,https://www.yelp.com/biz/sushi-masa-japanese-r...,145,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 34.8512725830078, 'longitude': -8...",[delivery],$$,"{'address1': '8590 Pelham Rd', 'address2': 'St...",+18642882227,(864) 288-2227,11481.830881
1,RGRk1ioORwm_FIX8PM732Q,konnichiwa-greenville,Konnichiwa,https://s3-media3.fl.yelpcdn.com/bphoto/FU6pBh...,False,https://www.yelp.com/biz/konnichiwa-greenville...,17,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.0,"{'latitude': 34.845952342825115, 'longitude': ...",[],NaN,"{'address1': '101 Falls Park Dr', 'address2': ...",+18642524436,(864) 252-4436,4184.255183
2,7cJxOV-ANX1qLThK3yV96w,otto-izakaya-greenville-4,Otto Izakaya,https://s3-media1.fl.yelpcdn.com/bphoto/TdPhFy...,False,https://www.yelp.com/biz/otto-izakaya-greenvil...,393,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 34.8228218820722, 'longitude': -8...",[delivery],$$,"{'address1': '15 Market Point Dr', 'address2':...",+18645688009,(864) 568-8009,5933.485357
3,VU0mq_4iDCZ3w1pDcbGiuQ,love-sushi-greenville-4,Love Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/zHiqdu...,False,https://www.yelp.com/biz/love-sushi-greenville...,14,"[{'alias': 'japanese', 'title': 'Japanese'}]",5.0,"{'latitude': 34.82095, 'longitude': -82.25788}",[],NaN,"{'address1': '2131 Woodruff Rd', 'address2': '...",+18646756360,(864) 675-6360,9540.226675
4,dvgaR-1sAfuHSi4_ZKXmwQ,sushi-murasaki-greenville,Sushi Murasaki,https://s3-media4.fl.yelpcdn.com/bphoto/sKCCZu...,False,https://www.yelp.com/biz/sushi-murasaki-greenv...,207,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 34.8506663, 'longitude': -82.3992...","[pickup, delivery]",$$,"{'address1': '2 S Main St', 'address2': '', 'a...",+18642712452,(864) 271-2452,3808.715842


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
94,xQZIvcjkH2R14yaHr2qQYQ,the-cheesecake-factory-greenville-greenville,The Cheesecake Factory - Greenville,https://s3-media2.fl.yelpcdn.com/bphoto/SUrPQg...,False,https://www.yelp.com/biz/the-cheesecake-factor...,400,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",3.0,"{'latitude': 34.8499166, 'longitude': -82.3335...","[delivery, restaurant_reservation]",$$,"{'address1': '700 Haywood Mall', 'address2': '...",+18642884444,(864) 288-4444,2209.333296
95,gUsB3gAE413Ezd3P6IxlmA,7-eleven-piedmont,7-Eleven,,False,https://www.yelp.com/biz/7-eleven-piedmont?adj...,5,"[{'alias': 'servicestations', 'title': 'Gas St...",2.0,"{'latitude': 34.6970574719195, 'longitude': -8...","[delivery, pickup]",$,"{'address1': '7803 Augusta Rd', 'address2': ''...",+18642772316,(864) 277-2316,17107.266905
96,_Ptzoh-8-8vJT_OF7t9TLQ,sprouts-farmers-market-simpsonville,Sprouts Farmers Market,https://s3-media3.fl.yelpcdn.com/bphoto/C1Xlae...,False,https://www.yelp.com/biz/sprouts-farmers-marke...,31,"[{'alias': 'healthmarkets', 'title': 'Health M...",4.0,"{'latitude': 34.8186242, 'longitude': -82.255452}",[],NaN,"{'address1': '2200 Woodruff Rd', 'address2': '...",+18647573240,(864) 757-3240,9866.418853
97,Ld7D2rUr627TNkoIdRkXwQ,lowes-foods-greenville-2,Lowes Foods,https://s3-media3.fl.yelpcdn.com/bphoto/3h-jGP...,False,https://www.yelp.com/biz/lowes-foods-greenvill...,5,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 34.86149155901255, 'longitude': -...",[],NaN,"{'address1': '3619 Pelham Rd', 'address2': '',...",+18642884162,(864) 288-4162,8304.264308
98,xb9QSdbk63Ani2-S5MrIHQ,harris-teeter-greenville-6,Harris Teeter,https://s3-media3.fl.yelpcdn.com/bphoto/ZelRSg...,False,https://www.yelp.com/biz/harris-teeter-greenvi...,24,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.5,"{'latitude': 34.8279736, 'longitude': -82.3987...",[],$$,"{'address1': '1720 Augusta St', 'address2': ''...",+18649778041,(864) 977-8041,4335.688854


In [15]:
# check for duplicate IDs
final_df.duplicated(subset='id').sum()

0

In [16]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_sushi.csv.gz', compression='gzip',index=False)